In [ ]:
from __future__ import division, print_function

# отключим всякие предупреждения Anaconda
import warnings
import os

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pylab as plt

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (6, 4)

**Скачиваем нужные csv-файлы с github**

Набор данных взят с сайта: 
https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

Набор данных взят с сайта: 
https://www.kaggle.com/quantbruce/real-estate-price-prediction

In [ ]:
if not os.path.exists('../data'): os.mkdir("../data")

!curl -o ../data/tr.csv https://raw.githubusercontent.com/NoxNokas/HomeWork/main/data/tr.csv

!curl -o ../data/RealEstate.csv https://raw.githubusercontent.com/NoxNokas/HomeWork/main/data/RealEstate.csv

# Классификация

## Предобработка и визуализация данных

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics, svm
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    KFold,
    cross_val_predict,
    cross_val_score,
    train_test_split,
)

In [ ]:
df = pd.read_csv("../data/tr.csv", names=["id", "diagnosis", "radius_mean", "texture_mean", "perimeter_mean", "area_mean",
                                      "smoothness_mean", "compactness_mean", "concavity_mean", "concave points_mean",
                                      "symmetry_mean", "fractal_dimension_mean", "radius_se", "texture_se",
                                      "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se",
                                      "concave points_se", "symmetry_se", "fractal_dimension_se", "radius_worst",
                                      "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
                                      "compactness_worst", "concavity_worst", "concave points_worst", "symmetry_worst",
                                      "fractal_dimension_worst"]).dropna()

In [ ]:
pd.concat([df.head(), df.tail()])

Информация об атрибутах:

1) Идентификационный номер

2) Диагноз (M = злокачественный, B = доброкачественный)

Для каждого ядра клетки вычисляются десять действительных признаков (в 3-32 столбцах):

* a) радиус (среднее расстояние от центра до точек по периметру)
* b) текстура (стандартное отклонение значений шкалы серого)
* c) периметр
* d) площадь
* e) гладкость (локальное изменение длины радиуса)
* f) компактность (периметр ^ 2 / площадь - 1,0)
* g) вогнутость (степень вогнутости частей контура)
* h ) вогнутые точки (количество вогнутых участков контура)
* i) симметрия
* j) фрактальная размерность («приближение береговой линии» - 1)

In [ ]:
df.info()

In [ ]:
sns.pairplot(df.loc[:,'diagnosis':'area_mean'], hue="diagnosis");

Видим, что целевой параметр в выборке распределён неравномерно

In [ ]:
sns.countplot(df["diagnosis"]);

In [ ]:
Y = df['diagnosis'].map({'M': 1, 'B': 0})
X = df.drop(['diagnosis', 'id'], axis=1)

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn", linewidths=.5, fmt= '.1f',ax=ax);

Видим, что radius_mean и perimeter_mean имеют идеальную корреляцию, и это не удивительно, т.к. с помощью радиуса вычисляется периметр. Найдём и уберём некоторые избыточные параметры.

In [ ]:
X = X.drop(["radius_mean", "perimeter_mean", "area_mean", "radius_se"], axis=1)

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn", linewidths=.5, fmt= '.1f',ax=ax);

Выделим 80% выборки (X_train, y_train) под обучение и 20% будут отложенной выборкой (X_test, y_test). отложенная выборка никак не будет участвовать в настройке параметров моделей, на ней мы в конце, после этой настройки, оценим качество полученной модели.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

Создаем и обучаем классификатор:
Основные параметры класса sklearn.neighbors.KNeighborsClassifier:


* weights: "uniform" (все веса равны), "distance" (вес обратно пропорционален расстоянию до тестового примера) или другая определенная пользователем функция 

* algorithm (опционально): "brute", "ball_tree", "KD_tree", или "auto". В первом случае ближайшие соседи для каждого тестового примера считаются перебором обучающей выборки. Во втором и третьем — расстояние между примерами хранятся в дереве, что ускоряет нахождение ближайших соседей. В случае указания параметра "auto" подходящий способ нахождения соседей будет выбран автоматически на основе обучающей выборки.

* leaf_size (опционально): порог переключения на полный перебор в случае выбора BallTree или KDTree для 
нахождения соседей

* metric: "minkowski", "manhattan", "euclidean", "chebyshev" и другие

## kNN

In [ ]:
knn_model = KNeighborsClassifier(
   n_neighbors=10, 
   weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=-1
)

In [ ]:
knn_model.fit(X_train, y_train)

In [ ]:
knn_pred = knn_model.predict(X_test)
print("Evaluation\t\t\tlabels 'M': 1, 'B': 0: \n\n", metrics.classification_report(y_test, knn_pred))

Используем скользящий контроль для поиска лучших параметров (англ. cross validation):

In [ ]:
model_params = knn_model.get_params()
tuned_params = {}
for k, v in model_params.items():
    tuned_params[k] = [v]
tuned_params['n_neighbors'] = list(range(1, 30))
clf = GridSearchCV(KNeighborsClassifier(), tuned_params, cv=10, n_jobs=-1)
clf.fit(X_train, y_train)
best_params = clf.best_params_

Оценка классификатора:

In [ ]:
knn_model = KNeighborsClassifier(**best_params)
knn_model.fit(X_train, y_train)
prediction1 = knn_model.predict(X_test)

In [ ]:
print('Used params:', best_params)
print("\nEvaluation\t\t\tlabels 'M': 1, 'B': 0: \n\n", metrics.classification_report(y_test, prediction1))

## Логистическая регрессия как линейный классификатор

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
prediction2 = model.predict(X_test)
print("The Logistic Regression is\n\n", metrics.classification_report(prediction2, y_test))

## SVM

In [ ]:
X_train, X_test, y_train, y_test
model = svm.SVC(kernel="linear", C=0.1, gamma=0.1)
model.fit(X_train, y_train)
prediction3 = model.predict(X_test)
print("The linear SVM is\n\n", metrics.classification_report(prediction3, y_test))

## Cross Validation

Мы должны обучить и протестировать наш алгоритм на каждом экземпляре набора данных. Затем мы можем взять среднее значение всех отмеченных погрешностей по набору данных.

Перекрестная проверка K-Fold разделяет набор данных на k-подмножества.

Допустим, мы делим набор данных на (k = 5) частей. Мы оставляем 1 часть для тестирования и обучаем алгоритм по оставшимся 4 частям.

Мы продолжаем процесс, меняя часть данных для тестирования на каждой итерации и обучая алгоритм по другим частям. Затем точность и ошибки усредняются, чтобы получить среднюю точность алгоритма.
Это называется перекрестной проверкой K-Fold.

Алгоритм может недообучаться на некоторых обучающих данных, а иногда переобучаться для другого обучающего набора. Таким образом, с помощью перекрестной проверки мы можем получить обобщенную модель.

In [ ]:
kfold = KFold(n_splits=10, random_state=22, shuffle=True)

classifiers = [
    "Linear Svm",
    "Logistic Regression",
    "KNN"
]
models = [
    svm.SVC(kernel="linear"),
    LogisticRegression(),
    KNeighborsClassifier(n_neighbors=9)
]

accuracy = []
for model in models:
    cv_result = cross_val_score(model, X, Y, cv=kfold, scoring="accuracy")
    accuracy.append(cv_result)

means = [cv_res.mean() for cv_res in accuracy]
stds = [cv_res.std() for cv_res in accuracy]

results_table = pd.DataFrame({"CV Mean": means, "Std": stds}, index=classifiers)
results_table.sort_values("CV Mean", ascending=False)

## Confusion Matrix

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(15, 4))

y_pred = cross_val_predict(svm.SVC(kernel="linear"), X, Y, cv=10)
sns.heatmap(metrics.confusion_matrix(Y, y_pred), ax=ax[0], annot=True, fmt="2.0f")
ax[0].set_title("Matrix for Linear-SVM")

y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9), X, Y, cv=10)
sns.heatmap(metrics.confusion_matrix(Y, y_pred), ax=ax[1], annot=True, fmt="2.0f")
ax[1].set_title("Matrix for KNN")


y_pred = cross_val_predict(LogisticRegression(), X, Y, cv=10)
sns.heatmap(metrics.confusion_matrix(Y, y_pred), ax=ax[2], annot=True, fmt="2.0f")
ax[2].set_title("Matrix for Logistic Regression");

Главная диагональ показывает количество правильных прогнозов, сделанных для каждого класса, а правая диагональ показывает количество сделанных неверных прогнозов.

# Регрессия

In [ ]:
# импортируем библиотеки
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LassoCV, RidgeCV # Cross Validation специальная для гребневой регуляризации и Лассо
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

## Предобработка и визуализация данных 

In [ ]:
df = pd.read_csv("../data/RealEstate.csv")
pd.concat([df.head(), df.tail()])

Описание параметров: Прогноз цен на недвижимость 

parameter | description 
--- | --- 
X1 | Дата транзакции.
X2 | Возраст дома.
X3 | Расстояние до ближайшей станции MRT.
X4 | Общее количество круглосуточных магазинов.
X5 | Широта.
X6 | Долгота.
**Y**  | Цена дома.

**Y** - это выход и, следовательно, зависимая переменная.

In [ ]:
df.info()

In [ ]:
# Проверяем корреляцию
df.corr()

In [ ]:
# Проверка на количество Null в столбцах
df.isnull().sum()

In [ ]:
# Удаляем столбцы с номером и датой транзакции
df.drop(["X1 transaction date", "No"], axis=1, inplace= True)

In [ ]:
sns.pairplot(df);

In [ ]:
# визуализирует корреляцию с помощью heatmap 
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(df.corr(), annot=True, cmap="RdYlGn", linewidths=.5, fmt= '.1f',ax=ax);

In [ ]:
# Визуализация результата набора данных обучения
sns.regplot(x="X2 house age", y="Y house price of unit area", data=df);

In [ ]:
sns.regplot(y="X3 distance to the nearest MRT station", x="Y house price of unit area", data=df);

In [ ]:
# Делим параметры на зависимые (y) и независимые (X)
X = df.loc[:,'X2 house age' : 'X6 longitude']
y = df.loc[:,'Y house price of unit area']

In [ ]:
# Разделяем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3,random_state = 0)

## Регрессия на основе k-ближайших соседей.

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train, y_train)

In [ ]:
print("MSE=%f" % np.sqrt(metrics.mean_squared_error(y_test, knn_model.predict(X_test))))

## Линейная регрессия

In [ ]:
# Обучаем модель
reg = LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
# Коэффициенты регрессии 
pd.DataFrame(reg.coef_, index=X.columns, columns=['coef']).sort_values(by=['coef'], ascending= False)

In [ ]:
print("MSE=%f" % np.sqrt(metrics.mean_squared_error(y_test, reg.predict(X_test))))

In [ ]:
# График прогнозов реальной цены и предсказанной
plt.scatter(y_test, reg.predict(X_test), color='cornflowerblue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.xlabel('Real Price')
plt.ylabel('Predicted')
plt.title('Прогнозирование линейной регрессии');

## SVR. Регрессия опорных векторов

In [ ]:
svr_model = make_pipeline(StandardScaler(), svm.SVR(C=2.5, epsilon=0.2))
svr_model.fit(X_train, y_train)

In [ ]:
print("MSE=%f" % np.sqrt(metrics.mean_squared_error(y_test, svr_model.predict(X_test))))

In [ ]:
# График прогнозов реальной цены и предсказанной
plt.scatter(y_test, svr_model.predict(X_test), color='cornflowerblue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.xlabel('Real Price')
plt.ylabel('Predicted')
plt.title('Прогнозирование линейной регрессии с помощью SVR');